In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, evaluate, dump
from surprise import SVD, SVDpp, KNNBaseline, KNNWithMeans, CoClustering
import threading
import thread


In [2]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)


In [3]:
def processDataFile(data,counter):
    df_nan = pd.DataFrame(pd.isnull(data.Rating))
    df_nan = df_nan[df_nan['Rating'] == True]
    df_nan = df_nan.reset_index()
    # print df_na
    movie_np = []
    movie_id = 1
    # print df_nan['index'][1:],df_nan['index'][:-1]
    a = zip(df_nan['index'][1:],df_nan['index'][:-1])
    store_df = pd.DataFrame([])
    temp_total_array = np.zeros(((0),1))
    for i,j in a:
        temp_np_array = np.full((i-j,1), counter)
        temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
        counter = counter+1
        #print counter
    #print temp_total_array.shape
    remaining =  data.shape[0]- temp_total_array.shape[0]
    temp_np_array = np.full((remaining,1), counter)
    temp_total_array =  np.concatenate((temp_total_array, temp_np_array))
    data['movie_id'] = temp_total_array
    data= data.dropna(thresh =3)
    return (data,counter)

In [4]:
total_data = pd.DataFrame([])

In [5]:
data = pd.read_csv("./input/combined_data_1.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,1)
#print "Counter",1
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Before (0, 0)
After (24053764, 4)


In [6]:
data = pd.read_csv("./input/combined_data_2.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape
#print mem_usage(data), mem_usage(total_data), " ", mem_usage(temp_data[0])

Counter 9209
Before (24053764, 4)
After (51031355, 4)


In [11]:
print "After",total_data.shape

After (73632984, 4)


In [9]:
data = pd.read_csv("./input/combined_data_3.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape

Counter 13365
Before (51031355, 4)
After (73632984, 4)


In [ ]:
total_data.shape

In [7]:
data = pd.read_csv("./input/combined_data_4.txt",  names = ['Cust_Id', 'Rating','date'])
temp_data = processDataFile(data,temp_data[1])
print "Counter",temp_data[1]
print "Before",total_data.shape
total_data = pd.concat([total_data,temp_data[0]],ignore_index=True)
print "After",total_data.shape


In [15]:
#del data
sample_df = total_data[:4000]

In [16]:
sample_df.head()

,Cust_Id,Rating,date,movie_id
0,1488844,3.0,2005-09-06,1.0
1,822109,5.0,2005-05-13,1.0
2,885013,4.0,2005-10-19,1.0
3,30878,4.0,2005-12-26,1.0
4,823519,3.0,2004-05-03,1.0


In [17]:
temp = pd.pivot_table(sample_df,index='Cust_Id', columns='movie_id')
# temp = pd.pivot_table(sample_df,'movie_id','Cust_Id'

### Create models as a dictionary below

In [12]:
coclustering = CoClustering()
knnbaseline = KNNBaseline()
knnwithmeans = KNNWithMeans()
svd = SVD()
svdpp = SVDpp()
modelList = {"coclustering" : coclustering, "knnbaseline" : knnbaseline, "knnwithmeans" : knnwithmeans, "svd" :svd, "svdpp" :svdpp}

In [39]:
reader = Reader()
                                                                                                                                                                                                                                                                                                                                                
# get just top 100K rows for faster run time
trainData = Dataset.load_from_df(total_data[['Cust_Id', 'movie_id', 'Rating']].sample(n=10000000), reader)
#data.split(n_folds=3)
#trainset = data.build_full_trainset()

In [40]:
trainData.split(n_folds=3)
trainset = trainData.build_full_trainset()

In [41]:
# Start of Probe Data manipulation 
probe = pd.read_csv("./input/probe.txt",  names = ['Cust_Id'])
# temp_data = processDataFile(probe,1)
probe_movie = pd.DataFrame([])
Cust_Id = []
movie_id = []
currentMovie =0
index = 0

for row in probe['Cust_Id']:
    if row[len(row)-1] == ":":
        currentMovie = int(row[:-1])
        #print currentMovie
        continue
    Cust_Id.append(row)
    movie_id.append(currentMovie)
    #print index,row
    index +=1
probe_movie['Cust_Id'] = Cust_Id
probe_movie['movie_id'] = movie_id
#print probe_movie[:30]
# test = total_data.loc[(total_data.Cust_Id.isin(probe_movie.Cust_Id)) & total_data.movie_id.isin(probe_movie.movie_id)]
merged_probe_set = pd.merge(probe_movie,total_data,on=['Cust_Id','movie_id'])
#print merged_probe_set[:30]
# End of Probe Data manipulation 

In [42]:
print len(probe)

1425333


In [44]:
def evaluateTrainAndTestModel(modelList, testData, predictions):
    # Evaluate model
    for model in modelList:
        print "Testing model: ", model
        try:
            evaluate(modelList[model], trainData, measures=['RMSE', 'MAE'])

            # Train model
            modelList[model].train(trainset)

            # Predict Movie ratings and compute error
            cust_id = [];
            pred = [];
            index =1;
            mae_sum = 0;
            mse_sum = 0;
            mse_sum_round = 0;
            rmse_round = 0;
            rmse = 0;
            n = len(merged_probe_set)
            for index,row in testData.iterrows():
                user = row['Cust_Id']
                currentMovie = row['movie_id']
                pred = modelList[model].predict(user, currentMovie, r_ui=row['Rating'], verbose=False)
                predictions[model][index]= pred.est
                diff = row['Rating'] - pred.est
                diffRound = row['Rating'] - round(pred.est)
                mse_sum += diff**2
                mse_sum_round += diffRound**2
                #print(diff)
            mse = mse_sum/n
            rmse = (mse_sum/n)**0.5
            rmse_round = (mse_sum_round/n)**0.5

            # TODO: Try using model.test
            #pred = model.test(testData['Cust_Id'])
            #rmse(pred)
            #dump.dump('./dump_SVD', pred, model)
            #print testData[:20]
            print "MSE: ", mse, " RMSE: ", rmse , " RMSE Roundup:  ", rmse_round
        except MemoryError:
            print "!!! Memory Error occured !!!"
            print "Continuing with next model"
    return predictions

In [45]:
predictions = pd.DataFrame(0, index=np.arange(len(merged_probe_set)), columns=modelList.keys())
evaluateTrainAndTestModel(modelList, merged_probe_set, predictions)


Testing model:  coclustering
Evaluating RMSE, MAE of algorithm CoClustering.

------------
Fold 1
RMSE: 0.9854
MAE:  0.7670
------------
Fold 2
RMSE: 0.9840
MAE:  0.7657
------------
Fold 3
RMSE: 0.9839
MAE:  0.7658
------------
------------
Mean RMSE: 0.9845
Mean MAE : 0.7662
------------
------------
    Cust_Id movie_id  Rating        date
0     30878        1     4.0  2005-12-26
1   2647871        1     4.0  2005-12-30
2   1283744        1     3.0  2004-04-17
3   2488120        1     5.0  2005-09-20
4    317050        1     5.0  2005-11-15
5   1904905        1     4.0  2005-05-13
6   1989766        1     4.0  2005-07-08
7     14756        1     4.0  2005-12-27
8   1027056        1     3.0  2005-12-03
9   1149588        1     4.0  2005-12-13
10  1394012        1     5.0  2005-12-19
11  1406595        1     4.0  2005-08-27
12  2529547        1     5.0  2005-11-18
13  1682104        1     4.0  2005-08-30
14  2625019        1     3.0  2005-09-12
15  2603381        1     5.0  2005-11-29

In [46]:
print predictions

        coclustering  knnbaseline  svd  knnwithmeans  svdpp
0                  3            0    0             0      0
1                  3            0    0             0      0
2                  3            0    0             0      0
3                  4            0    0             0      0
4                  4            0    0             0      0
5                  4            0    0             0      0
6                  3            0    0             0      0
7                  3            0    0             0      0
8                  2            0    0             0      0
9                  3            0    0             0      0
10                 3            0    0             0      0
11                 3            0    0             0      0
12                 3            0    0             0      0
13                 4            0    0             0      0
14                 3            0    0             0      0
15                 4            0    0  

***Data Munging***
<br> The data is not available as a csv, instead it is a TXT file. There are four files which contain the customer ids and the rating that they gave to the movies. In each of these files there are rows with a number followed by a colon and  the rows that follow this kind of rows have the customer ids with the rating and the date on which the rating was given.
To organize this data we have done the following steps:<br>
1. We find all the indices which have movie ids in them and store them in an array.
2. We zip this array with itself such that the resulting dataframe contains the value of the next index in the second column.
3. We use the resultant dataframe and use the values in the first 2 columns to fill an array with movie ids.
4. We append the new array to dataframe that we obtained from the TXT files.
5. Finally, we drop those rows which have the movie ids with colons in them.

We named this dataframe as total_data.

Similarly for the probe set, we read the probe.txt file and store it in a dataframe. Just like we cleaned the movie data set as mentioned above, we have cleaned the probe data as well. The problem with probe data set is that the rows dont have the actual ratings that the movies got. So, we had to find the actual ratings from total_data dataframe by merging the probe dataframe with the total_data dataframe. We named this dataframe as merged_probe_set.<br>

In this assignment we have used collaborative learning as our modelling method for unsupervised learning.


**What is Collaborative Filtering?**
<br>Collaborative Filtering uses preferences of other users to determine the preference of the user in consideration. For example, if person A likes Movie 1,2 and 3, and person B likes movies 1 and 3, then there is a high probability that Person A and B will have affinity for the same kind of movies. So, we can safely assume that Person A and Person B are liked minded and we can use one person's rating for a particular movie to determine the other person's liking for that movie.
<br>
Formally, Collaborative filtering is the process of filtering for information or patterns using techniques involving collaboration among multiple agents, viewpoints, data sources, etc. <a href="https://en.wikipedia.org/wiki/Collaborative_filtering">Source</a><br>

**We have used the following clustering techniques with their corresponding MSE and RMSE on the probe set: **<br>

1. CoClustering: In Co-Clustering, the users are assigned some clusters and some co-clusters. We calculate the predicion using the sum of the difference of the mean of individual clusters and the difference of the co-clusters.
MSE:  1.28873601364  RMSE:  1.13522509382
2. SVD: the user-rating matrix of movies after an SVD, will decompose into matrices that represents latent user-user features and item-item features, e.g. same gender user, same age-group of users, action movies etc. and many other latent factors involved in the rating behavior that is not apparent from the user-rating matrix.<a href ="https://www.quora.com/Whats-the-difference-between-SVD-and-SVD++">Source</a>
3. SVDpp: This is similar to SVD except, it factors in item-item neighborhood and user-user neighborhood models.

**Other Methods Used**
1. The basic model which we built for this dataset was Linear Regression. The accuracy for prediction of the movie ratings in the probe dataset was sufficiently high, as close as 1.1301 over the entire dataset. However, linear regression is not recommended for such a dataset as this is specifically designed for prediction of ratings either by pattern finding or clustering.

2. We tried converting the given problem into a supervised learning one. We divided the ratings into 5 classes namely 1 to 5, and then tried predicting the classes for a small dataset. The data being skewed and unsuitable for classification due to lack of features, it almost always failed.

**Prevention of Over-fitting**

Although overfitting is not so common in Unsupervised learning as it is in Supervised learing but there are a certain scenarios where overfitting can occur in Unsupervised learning as well. For example: In the following two scenarios:
1. There are N elements and N cluster, and each element belongs to its own cluster.
2. On the other hand if there is one big cluster which has all the elements. Then also the error rate would be very high.

The module that we used to implement collaborative filtering has a feature in which the training data is divided into folds. The api divides the data into a certain number of folds, lets say 5, and out of these 5 folds it uses 4 folds to train and 1 fold to validate the predictions. This helps us in keeping the overfitting in check.

**Hyper-parameter Tuning**
We implemented parameter tuning for SVD and SVDpp models to try the performance change. We used the GridSearch class of Scikit_Surprise for this purpose. We tried setting different n_epoch values for these models and figured out that the accuracy was better when this value is close to 5.

**Results**
Following are the MSE and RMSE for the Models that we ran:

1. CoClustering: MSE:  1.13283932008  RMSE:  1.06434924723  RMSE Roundup:   1.10445979131
2. KNNBaseline: Memory Error
3. SVD: MSE:  1.1314760371  RMSE:  1.06370862416  RMSE Roundup:   1.10322495319
4. SVDpp: MSE:  1.0972537568 RMSE: 1.04749880993
5. Linear Regression: RMSE: 1.130103297749197

<br>
***Individual Contributions:***
<br>
Ishupreet Singh 111424300  <br> 
Bhushan Sonawane 111511679 <br>
Nishant Borude 111447198 <br>
Our primary motive from this assignment was learning. So, we did not divide the assignment amongst each other instead all three of us individualy and separately did the assignment and merged the best of all three. <br>

Overall, we believe that the contribution of all members of the team was equal.

***Issues Faced***
Due to lack of resources most of the highly accurate prediction models such as KNNBaseline, KNNWithMeans would crash with memory error even on 16GB RAM. We are currently implementing our solution for memory intensive models on Google Cloud Platform which might take some time for coming up with an updated report.

***References***
https://www.kaggle.com/laowingkin/netflix-movie-recommendation
Followed the kernel provided by D. Lao on the Netflix Data Challenge page for data preprocessing.